##### Copyright 2020 The TensorFlow IO Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow를 사용한 Azure Blob 저장소

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/io/tutorials/azure"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a>   </td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/io/blob/master/docs/tutorials/azure.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/io/blob/master/docs/tutorials/azure.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">깃허브(GitHub) 소스 보기</a>
</td>
      <td><a href="https://storage.googleapis.com/tensorflow_docs/io/docs/tutorials/azure.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
</table>

주의: 이 노트북은 Python 패키지 외에도 `npm install --user`를 사용하여 패키지를 설치합니다. 로컬에서 실행할 때 주의하세요.


## 개요

이 튜토리얼에서는 TensorFlow IO의 Azure 파일 시스템 통합을 통해 TensorFlow와 함께 [Azure Blob 저장소](https://azure.microsoft.com/en-us/services/storage/blobs/)에서 파일 읽기 및 쓰기를 사용하는 방법을 보여줍니다.

Azure Blob 저장소에서 파일을 읽고 쓰려면 Azure 저장소 계정이 필요합니다. Azure Storage Key는 환경 변수를 통해 제공되어야 합니다.

```
os.environ['TF_AZURE_STORAGE_KEY'] = '&lt;key&gt;'
```

저장소 계정 이름과 컨테이너 이름은 파일 이름 uri의 일부입니다.

```
azfs://&lt;storage-account-name&gt;/&lt;container-name&gt;/&lt;path&gt;
```

이 튜토리얼에서는 데모 목적으로 Azure 저장소 에뮬레이터인 [Azurite](https://github.com/Azure/Azurite)를 선택적으로 설정할 수 있습니다. Azurite 에뮬레이터에서는 TensorFlow를 사용하여 Azure Blob 저장소 인터페이스를 통해 파일을 읽고 쓸 수 있습니다.

## 설정 및 사용

### 필수 패키지를 설치하고 런타임 다시 시작하기

In [2]:
try:
  %tensorflow_version 2.x 
except Exception:
  pass

!pip install tensorflow-io

TensorFlow 2.x selected.
     |████████████████████████████████| 20.1MB 42.7MB/s 


### Azurite 설치 및 설정하기(선택 사항)

Azure 저장소 계정을 사용할 수 없는 경우 Azure 저장소 인터페이스를 에뮬레이트하는 Azurite를 설치하고 설정하기 위해 다음이 필요합니다.

In [3]:
!npm install azurite@2.7.0

npm WARN deprecated request@2.87.0: request has been deprecated, see https://github.com/request/request/issues/3142
npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ azurite@2.7.0
added 116 packages from 141 contributors in 6.591s


In [4]:
# The path for npm might not be exposed in PATH env,
# you can find it out through 'npm bin' command
npm_bin_path = get_ipython().getoutput('npm bin')[0]
print('npm bin path: ', npm_bin_path)

# Run `azurite-blob -s` as a background process. 
# IPython doesn't recognize `&` in inline bash cells.
get_ipython().system_raw(npm_bin_path + '/' + 'azurite-blob -s &')

npm bin path:  /content/node_modules/.bin


### TensorFlow를 사용하여 Azure 저장소에 파일 읽기 및 쓰기

다음은 TensorFlow의 API를 사용하여 Azure 저장소에서 파일을 읽고 쓰는 예입니다.

`tensorflow-io` 패키지를 가져오면 `tensorflow-io`가 사용할 `azfs`를 자동으로 등록하므로 TensorFlow의 다른 파일 시스템(예: POSIX 또는 GCS)과 동일한 방식으로 동작합니다.

Azure 저장소 키는 `TF_AZURE_STORAGE_KEY` 환경 변수를 통해 제공되어야 합니다. 그렇지 않으면 `TF_AZURE_USE_DEV_STORAGE`를 `True`로 설정하여 Azurite 에뮬레이터를 대신 사용할 수 있습니다.


In [ ]:
import os
import tensorflow as tf
import tensorflow_io as tfio

# Switch to False to use Azure Storage instead:
use_emulator = True

if use_emulator:
  os.environ['TF_AZURE_USE_DEV_STORAGE'] = '1'
  account_name = 'devstoreaccount1'
else:
  # Replace <key> with Azure Storage Key, and <account> with Azure Storage Account
  os.environ['TF_AZURE_STORAGE_KEY'] = '<key>'
  account_name = '<account>'

  # Alternatively, you can use a shared access signature (SAS) to authenticate with the Azure Storage Account
  os.environ['TF_AZURE_STORAGE_SAS'] = '<your sas>'
  account_name = '<account>'

In [6]:
pathname = 'az://{}/aztest'.format(account_name)
tf.io.gfile.mkdir(pathname)

filename = pathname + '/hello.txt'
with tf.io.gfile.GFile(filename, mode='w') as w:
  w.write("Hello, world!")

with tf.io.gfile.GFile(filename, mode='r') as r:
  print(r.read())

Hello, world!


## 구성

TensorFlow에서 Azure Blob 저장소의 구성은 항상 환경 변수를 통해 수행됩니다. 다음은 사용 가능한 구성의 전체 목록입니다.

- `TF_AZURE_USE_DEV_STORAGE`: 'az://devstoreaccount1/container/file.txt'와 같은 연결에 로컬 개발 저장소 에뮬레이터를 사용하려면 1로 설정합니다. 다른 모든 설정보다 우선하므로 다른 연결을 사용하도록 `unset`할 수 있습니다.
- `TF_AZURE_STORAGE_KEY`: 사용 중인 저장소 계정의 계정 키입니다.
- `TF_AZURE_STORAGE_USE_HTTP`: https 전송을 사용하지 않으려면 아무 값으로나 설정합니다. https 기본값을 사용하려면 `unset`을 사용합니다.
- `TF_AZURE_STORAGE_BLOB_ENDPOINT`: Blob 저장소의 엔드포인트로 설정합니다. 기본값은 `.core.windows.net`입니다.
